In [1]:
# Imports
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import itertools
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
import pickle

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Modeling

In this notebook, we are taking the output from 3_EDA to start doing the modeling. To recap on the missing steps, we will:

#### `3. Model Selection`
Given the nature the data, we might consider the following models:
- **ARIMA/SARIMA**: If your data shows trends or autocorrelation. SARIMA is suitable if there is a clear seasonal pattern.
- **Prophet**: Handles daily data well, robust to missing data, and good with seasonality.
- **Machine Learning Approaches**: If there are other factors that can predict 'quantity', a model like Random Forest or Gradient Boosting might be useful.

#### 4. Model Training and Forecasting
- **Training**: Training the yearly data.
- **Forecasting**: Generate predictions for the upcomming days.
- **Hyperparameter Tuning**: Optimize model parameters for best performance.

#### 5. Model Evaluation
- **Performance Metrics**: Evaluate the model using appropriate metrics.
- **Cross-Validation**: If possible, use time series cross-validation.

Here's a general approach to fitting an ARIMA model:

Selecting Model Orders: You need to choose the SARIMA model orders. These include the non-seasonal parameters (p, d, q) and the seasonal. Since you've detrended the data, d might be 0 or 1 depending on if the data needs further differencing to achieve stationarity. The seasonal parameters will incorporate the seasonality you've observed.

Seasonal Period (s): This is the seasonality of the data. In your case, if the seasonality is twice per month, you've already calculated the seasonal period to be approximately 365 hours.

Parameter Estimation: Use the ACF and PACF plots of the detrended data to estimate the initial ARIMA parameters. You can also use grid search methods to test different combinations of parameters and select the best model based on some criteria like the AIC (Akaike Information Criterion).

Model Diagnostics: After fitting the model, check the diagnostics to ensure that the residuals of your model are white noise (no autocorrelation).

# Modeling by Country

We will do a separate model for each country taking the signla processed from the last EDA step.

In [2]:
# Parsing date strings, ignoring any timezone information and converting them to datetime objects
date_parser = lambda x: pd.to_datetime(x[:22])
forecasted_values_dict = {}

## SPAIN Model

In [3]:
# Reading the dataset Spain
model_spain_df = pd.read_csv("../3_EDA/EDA_countries/EDA_SP.csv", 
                     converters={'EndTime': date_parser}).set_index('EndTime').dropna()

Initial Model

In [4]:
warnings.filterwarnings("ignore")

p = 1  # replace with actual
d = 1  # replace with actual
q = 1  # replace with actual

# Fit the ARIMA model (using the known part of the time series)
model = ARIMA(model_spain_df['detrended'], order=(p, d, q))
model_fit = model.fit()

# Forecast the missing values
# The 'steps' argument would be the number of hours in the missing months
forecast = model_fit.get_forecast(steps=1000) #hours in a monyh

# The forecast object contains the predicted values and other information
forecasted_values = forecast.predicted_mean

# You may also want to extract the confidence intervals of the forecasts
conf_int = forecast.conf_int()

Hyperparameter Tunning and Train / Test Split

In [5]:
warnings.filterwarnings("ignore")

# Split the dataset into train and test sets (80-20 split)
train_size = int(len(model_spain_df) * 0.8)
train, test = model_spain_df.iloc[:train_size], model_spain_df.iloc[train_size:]

# Define the p, d, and q ranges
p = d = q = range(0, 3)  # Example ranges; adjust as needed

# Generate all different combinations of p, d, and q triplets
pdq = list(itertools.product(p, d, q))

# Initialize best score and parameters
best_aic = float("inf")
best_order = None
best_model = None

# Grid search for the best ARIMA model on the training set
for order in pdq:
    try:
        model = ARIMA(train['detrended'], order=order)
        model_fit = model.fit()
        if model_fit.aic < best_aic:
            best_aic = model_fit.aic
            best_order = order
            best_model = model_fit
    except:
        continue

print(f'Best ARIMA{best_order} AIC: {best_aic}')

# Forecasting on the test set
forecast = best_model.get_forecast(steps=len(test))  # Forecasting as many steps as the test set
forecasted_values = forecast.predicted_mean

# Compute evaluation metrics
mse = mean_squared_error(test['detrended'], forecasted_values)
mae = mean_absolute_error(test['detrended'], forecasted_values)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

Best ARIMA(2, 1, 2) AIC: 123821.37328905759
Mean Squared Error: 264054434.661867
Mean Absolute Error: 13138.66812549769


Saving Forecasted Value

In [6]:
# Saving value in dict
forecasted_values_dict['Spain'] = forecasted_values.iloc[0]

Saving model in pickle file

In [7]:
# Save the best model to a file
with open('../models/best_arima_model_SP.pkl', 'wb') as file:
    pickle.dump(best_model, file)

## POLAND Model

In [10]:
# Reading the dataset Poland
model_poland_df = pd.read_csv("../3_EDA/EDA_countries/EDA_PO.csv", 
                     converters={'EndTime': date_parser}).set_index('EndTime').dropna()

# Split the dataset into train and test sets (80-20 split)
train_size = int(len(model_poland_df) * 0.8)
train, test = model_poland_df.iloc[:train_size], model_poland_df.iloc[train_size:]

# Define the p, d, and q ranges
p = d = q = range(0, 3)  # Example ranges; adjust as needed

# Generate all different combinations of p, d, and q triplets
pdq = list(itertools.product(p, d, q))

# Initialize best score and parameters
best_aic = float("inf")
best_order = None
best_model = None

# Grid search for the best ARIMA model on the training set
for order in pdq:
    try:
        model = ARIMA(train['detrended'], order=order)
        model_fit = model.fit()
        if model_fit.aic < best_aic:
            best_aic = model_fit.aic
            best_order = order
            best_model = model_fit
    except:
        continue

print(f'Best ARIMA{best_order} AIC: {best_aic}')

# Forecasting on the test set
forecast = best_model.get_forecast(steps=len(test))  # Forecasting as many steps as the test set
forecasted_values = forecast.predicted_mean

# Compute evaluation metrics
mse = mean_squared_error(test['detrended'], forecasted_values)
mae = mean_absolute_error(test['detrended'], forecasted_values)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

# Saving value in dict
forecasted_values_dict['Poland'] = forecasted_values.iloc[0]

# Save the best model to a file
with open('../models/best_arima_model_PO.pkl', 'wb') as file:
    pickle.dump(best_model, file)

Best ARIMA(2, 1, 2) AIC: 101967.019733206
Mean Squared Error: 9385116.075622112
Mean Absolute Error: 2629.4588517453353


## DENMARK Model

In [11]:
# Reading the dataset Denmark
model_denmark_df = pd.read_csv("../3_EDA/EDA_countries/EDA_DK.csv", 
                     converters={'EndTime': date_parser}).set_index('EndTime').dropna()

# Split the dataset into train and test sets (80-20 split)
train_size = int(len(model_denmark_df) * 0.8)
train, test = model_denmark_df.iloc[:train_size], model_denmark_df.iloc[train_size:]

# Define the p, d, and q ranges
p = d = q = range(0, 3)  # Example ranges; adjust as needed

# Generate all different combinations of p, d, and q triplets
pdq = list(itertools.product(p, d, q))

# Initialize best score and parameters
best_aic = float("inf")
best_order = None
best_model = None

# Grid search for the best ARIMA model on the training set
for order in pdq:
    try:
        model = ARIMA(train['detrended'], order=order)
        model_fit = model.fit()
        if model_fit.aic < best_aic:
            best_aic = model_fit.aic
            best_order = order
            best_model = model_fit
    except:
        continue

print(f'Best ARIMA{best_order} AIC: {best_aic}')

# Forecasting on the test set
forecast = best_model.get_forecast(steps=len(test))  # Forecasting as many steps as the test set
forecasted_values = forecast.predicted_mean

# Compute evaluation metrics
mse = mean_squared_error(test['detrended'], forecasted_values)
mae = mean_absolute_error(test['detrended'], forecasted_values)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

# Saving value in dict
forecasted_values_dict['Poland'] = forecasted_values.iloc[0]

# Save the best model to a file
with open('../models/best_arima_model_DK.pkl', 'wb') as file:
    pickle.dump(best_model, file)

Best ARIMA(2, 0, 2) AIC: 92253.64272046028
Mean Squared Error: 1709821.646026413
Mean Absolute Error: 1071.8352269360382


## SWEEDEN Model

In [12]:
# Reading the dataset Sweeden
model_sweeden_df = pd.read_csv("../3_EDA/EDA_countries/EDA_SE.csv", 
                     converters={'EndTime': date_parser}).set_index('EndTime').dropna()

# Split the dataset into train and test sets (80-20 split)
train_size = int(len(model_sweeden_df) * 0.8)
train, test = model_sweeden_df.iloc[:train_size], model_sweeden_df.iloc[train_size:]

# Define the p, d, and q ranges
p = d = q = range(0, 3)  # Example ranges; adjust as needed

# Generate all different combinations of p, d, and q triplets
pdq = list(itertools.product(p, d, q))

# Initialize best score and parameters
best_aic = float("inf")
best_order = None
best_model = None

# Grid search for the best ARIMA model on the training set
for order in pdq:
    try:
        model = ARIMA(train['detrended'], order=order)
        model_fit = model.fit()
        if model_fit.aic < best_aic:
            best_aic = model_fit.aic
            best_order = order
            best_model = model_fit
    except:
        continue

print(f'Best ARIMA{best_order} AIC: {best_aic}')

# Forecasting on the test set
forecast = best_model.get_forecast(steps=len(test))  # Forecasting as many steps as the test set
forecasted_values = forecast.predicted_mean

# Compute evaluation metrics
mse = mean_squared_error(test['detrended'], forecasted_values)
mae = mean_absolute_error(test['detrended'], forecasted_values)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

# Saving value in dict
forecasted_values_dict['Sweeden'] = forecasted_values.iloc[0]

# Save the best model to a file
with open('../models/best_arima_model_SE.pkl', 'wb') as file:
    pickle.dump(best_model, file)

Best ARIMA(2, 0, 2) AIC: 97241.05130632836
Mean Squared Error: 1784595.9373301351
Mean Absolute Error: 1025.10073167753


## NETHERLANDS Model

In [13]:
# Reading the dataset Netherlands
model_ned_df = pd.read_csv("../3_EDA/EDA_countries/EDA_NE.csv", 
                     converters={'EndTime': date_parser}).set_index('EndTime').dropna()

# Split the dataset into train and test sets (80-20 split)
train_size = int(len(model_ned_df) * 0.8)
train, test = model_ned_df.iloc[:train_size], model_ned_df.iloc[train_size:]

# Define the p, d, and q ranges
p = d = q = range(0, 3)  # Example ranges; adjust as needed

# Generate all different combinations of p, d, and q triplets
pdq = list(itertools.product(p, d, q))

# Initialize best score and parameters
best_aic = float("inf")
best_order = None
best_model = None

# Grid search for the best ARIMA model on the training set
for order in pdq:
    try:
        model = ARIMA(train['detrended'], order=order)
        model_fit = model.fit()
        if model_fit.aic < best_aic:
            best_aic = model_fit.aic
            best_order = order
            best_model = model_fit
    except:
        continue

print(f'Best ARIMA{best_order} AIC: {best_aic}')

# Forecasting on the test set
forecast = best_model.get_forecast(steps=len(test))  # Forecasting as many steps as the test set
forecasted_values = forecast.predicted_mean

# Compute evaluation metrics
mse = mean_squared_error(test['detrended'], forecasted_values)
mae = mean_absolute_error(test['detrended'], forecasted_values)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

# Saving value in dict
forecasted_values_dict['Netherlands'] = forecasted_values.iloc[0]

# Save the best model to a file
with open('../models/best_arima_model_NE.pkl', 'wb') as file:
    pickle.dump(best_model, file)

Best ARIMA(2, 1, 2) AIC: 115764.7337641639
Mean Squared Error: 72803800.19662891
Mean Absolute Error: 6896.542038156176


## GERMANY Model

In [14]:
# Reading the dataset Denmark
model_ger_df = pd.read_csv("../3_EDA/EDA_countries/EDA_DE.csv", 
                     converters={'EndTime': date_parser}).set_index('EndTime').dropna()

# Split the dataset into train and test sets (80-20 split)
train_size = int(len(model_ger_df) * 0.8)
train, test = model_ger_df.iloc[:train_size], model_ger_df.iloc[train_size:]

# Define the p, d, and q ranges
p = d = q = range(0, 3)  # Example ranges; adjust as needed

# Generate all different combinations of p, d, and q triplets
pdq = list(itertools.product(p, d, q))

# Initialize best score and parameters
best_aic = float("inf")
best_order = None
best_model = None

# Grid search for the best ARIMA model on the training set
for order in pdq:
    try:
        model = ARIMA(train['detrended'], order=order)
        model_fit = model.fit()
        if model_fit.aic < best_aic:
            best_aic = model_fit.aic
            best_order = order
            best_model = model_fit
    except:
        continue

print(f'Best ARIMA{best_order} AIC: {best_aic}')

# Forecasting on the test set
forecast = best_model.get_forecast(steps=len(test))  # Forecasting as many steps as the test set
forecasted_values = forecast.predicted_mean

# Compute evaluation metrics
mse = mean_squared_error(test['detrended'], forecasted_values)
mae = mean_absolute_error(test['detrended'], forecasted_values)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

# Saving value in dict
forecasted_values_dict['Germany'] = forecasted_values.iloc[0]

# Save the best model to a file
with open('../models/best_arima_model_DE.pkl', 'wb') as file:
    pickle.dump(best_model, file)

Best ARIMA(2, 1, 2) AIC: 135040.59889130667
Mean Squared Error: 1310454216.1989555
Mean Absolute Error: 29935.81739021916


## UK Model

In [15]:
# Reading the dataset UK
model_uk_df = pd.read_csv("../3_EDA/EDA_countries/EDA_UK.csv", 
                     converters={'EndTime': date_parser}).set_index('EndTime').dropna()

# Split the dataset into train and test sets (80-20 split)
train_size = int(len(model_uk_df) * 0.8)
train, test = model_uk_df.iloc[:train_size], model_uk_df.iloc[train_size:]

# Define the p, d, and q ranges
p = d = q = range(0, 3)  # Example ranges; adjust as needed

# Generate all different combinations of p, d, and q triplets
pdq = list(itertools.product(p, d, q))

# Initialize best score and parameters
best_aic = float("inf")
best_order = None
best_model = None

# Grid search for the best ARIMA model on the training set
for order in pdq:
    try:
        model = ARIMA(train['detrended'], order=order)
        model_fit = model.fit()
        if model_fit.aic < best_aic:
            best_aic = model_fit.aic
            best_order = order
            best_model = model_fit
    except:
        continue

print(f'Best ARIMA{best_order} AIC: {best_aic}')

# Forecasting on the test set
forecast = best_model.get_forecast(steps=len(test))  # Forecasting as many steps as the test set
forecasted_values = forecast.predicted_mean

# Compute evaluation metrics
mse = mean_squared_error(test['detrended'], forecasted_values)
mae = mean_absolute_error(test['detrended'], forecasted_values)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

# Saving value in dict
forecasted_values_dict['UK'] = forecasted_values.iloc[0]

# Save the best model to a file
with open('../models/best_arima_model_UK.pkl', 'wb') as file:
    pickle.dump(best_model, file)

Best ARIMA(2, 1, 2) AIC: 13860.657872593514
Mean Squared Error: 277639.4118961378
Mean Absolute Error: 421.40105735093755


## ITALY Model

In [16]:
# Reading the dataset Italy
model_it_df = pd.read_csv("../3_EDA/EDA_countries/EDA_IT.csv", 
                     converters={'EndTime': date_parser}).set_index('EndTime').dropna()

# Split the dataset into train and test sets (80-20 split)
train_size = int(len(model_it_df) * 0.8)
train, test = model_it_df.iloc[:train_size], model_it_df.iloc[train_size:]

# Define the p, d, and q ranges
p = d = q = range(0, 3)  # Example ranges; adjust as needed

# Generate all different combinations of p, d, and q triplets
pdq = list(itertools.product(p, d, q))

# Initialize best score and parameters
best_aic = float("inf")
best_order = None
best_model = None

# Grid search for the best ARIMA model on the training set
for order in pdq:
    try:
        model = ARIMA(train['detrended'], order=order)
        model_fit = model.fit()
        if model_fit.aic < best_aic:
            best_aic = model_fit.aic
            best_order = order
            best_model = model_fit
    except:
        continue

print(f'Best ARIMA{best_order} AIC: {best_aic}')

# Forecasting on the test set
forecast = best_model.get_forecast(steps=len(test))  # Forecasting as many steps as the test set
forecasted_values = forecast.predicted_mean

# Compute evaluation metrics
mse = mean_squared_error(test['detrended'], forecasted_values)
mae = mean_absolute_error(test['detrended'], forecasted_values)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

# Saving value in dict
forecasted_values_dict['Italy'] = forecasted_values.iloc[0]

# Save the best model to a file
with open('../models/best_arima_model_IT.pkl', 'wb') as file:
    pickle.dump(best_model, file)

Best ARIMA(2, 1, 1) AIC: 62189.27592911603
Mean Squared Error: 44210889.877709255
Mean Absolute Error: 5422.750973369246


## HUNGARY Model

In [17]:
# Reading the dataset Hungary
model_hun_df = pd.read_csv("../3_EDA/EDA_countries/EDA_HU.csv", 
                     converters={'EndTime': date_parser}).set_index('EndTime').dropna()

# Split the dataset into train and test sets (80-20 split)
train_size = int(len(model_hun_df) * 0.8)
train, test = model_hun_df.iloc[:train_size], model_hun_df.iloc[train_size:]

# Define the p, d, and q ranges
p = d = q = range(0, 3)  # Example ranges; adjust as needed

# Generate all different combinations of p, d, and q triplets
pdq = list(itertools.product(p, d, q))

# Initialize best score and parameters
best_aic = float("inf")
best_order = None
best_model = None

# Grid search for the best ARIMA model on the training set
for order in pdq:
    try:
        model = ARIMA(train['detrended'], order=order)
        model_fit = model.fit()
        if model_fit.aic < best_aic:
            best_aic = model_fit.aic
            best_order = order
            best_model = model_fit
    except:
        continue

print(f'Best ARIMA{best_order} AIC: {best_aic}')

# Forecasting on the test set
forecast = best_model.get_forecast(steps=len(test))  # Forecasting as many steps as the test set
forecasted_values = forecast.predicted_mean

# Compute evaluation metrics
mse = mean_squared_error(test['detrended'], forecasted_values)
mae = mean_absolute_error(test['detrended'], forecasted_values)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

# Saving value in dict
forecasted_values_dict['Hungary'] = forecasted_values.iloc[0]

# Save the best model to a file
with open('../models/best_arima_model_HU.pkl', 'wb') as file:
    pickle.dump(best_model, file)

Best ARIMA(2, 1, 2) AIC: 103541.18900170678
Mean Squared Error: 7815637.281220619
Mean Absolute Error: 2372.265075446539


## Forecasted Country with Greater Value

In [19]:
forecasted_values_dict

{'Spain': 21042.00489669611,
 'Poland': -1039.8978309942197,
 'Sweeden': -97.33877070294022,
 'Netherlands': 1281.8994967560648,
 'Germany': 19194.084806305484,
 'UK': 781.5245527120798,
 'Italy': 5424.729903671684,
 'Hungary': 630.0741785469036}

In [18]:
# Find the country with the highest forecasted value
country_with_max_value = max(forecasted_values_dict, key=forecasted_values_dict.get)

print(f"The country with the greatest forecasted value is: {country_with_max_value}")

The country with the greatest forecasted value is: Spain
